In [18]:
import pandas as pd
import numpy as np

from keras import optimizers, losses, activations, models
from keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler, ReduceLROnPlateau
from keras.layers import Dense, Input, Dropout, Convolution1D, MaxPool1D, GlobalMaxPool1D, GlobalAveragePooling1D, \
    concatenate, LSTM, Reshape, Bidirectional, Masking
from keras.models import Sequential
from sklearn.metrics import f1_score, accuracy_score

# Need the same seeds etc as baseline


# df_train = pd.read_csv("../input/mitbih_train.csv", header=None)
df_train = pd.read_csv("../input/mitbih_train.csv", header=None)
df_train = df_train.sample(frac=1)
# df_test = pd.read_csv("../input/mitbih_test.csv", header=None)
df_test = pd.read_csv("../input/mitbih_test.csv", header=None)

Y = np.array(df_train[187].values).astype(np.int8)
X = np.array(df_train[list(range(187))].values)[..., np.newaxis]

Y_test = np.array(df_test[187].values).astype(np.int8)
X_test = np.array(df_test[list(range(187))].values)[..., np.newaxis]

BATCH_SIZE = 512


def build_model():
    nclass = 5
    dr = 0.2
    rec_dr = 0
    # create model
    model = Sequential()
    # model.add(Masking(mask_value=0., input_shape=(187, 1)))
    model.add(Bidirectional(LSTM(128, dropout=dr, recurrent_dropout=rec_dr, return_sequences=True), input_shape=(187, 1)))
    model.add(Bidirectional(LSTM(64, dropout=dr, recurrent_dropout=rec_dr, return_sequences=True)))
    model.add(Bidirectional(LSTM(32, dropout=dr, recurrent_dropout=rec_dr)))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(nclass, activation='softmax'))
    model.compile(loss='sparse_categorical_crossentropy',
                  optimizer='nadam', metrics=['acc'])  #  weighted_metrics=['accuracy']
    # model.build((187, 1)) # if do not give input shape in first layer can call build with input shape (here cause commented masking)
    model.summary()
    return model

model = build_model()
file_path = "BiLSTM_mitbih.h5"
checkpoint = ModelCheckpoint(file_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
early = EarlyStopping(monitor="val_acc", mode="max", patience=3, verbose=1)
# TODO reduce LR on Plateau useful with nadam ? doesn't it already has a momentum going on ?
#  adam is RMSprop with momentum here just used nesterov momentum
redonplat = ReduceLROnPlateau(monitor="val_acc", mode="max", patience=3, verbose=2)
callbacks_list = [checkpoint, early, redonplat]  # early

is_trained = False
if not is_trained:
    model.fit(X, Y, epochs=20, batch_size=BATCH_SIZE, verbose=1, callbacks=callbacks_list, validation_split=0.1)

model.load_weights(file_path)

pred_test = model.predict(X_test)
pred_test = np.argmax(pred_test, axis=-1)

f1 = f1_score(Y_test, pred_test, average="macro")  # average c

print("Test f1 score : %s " % f1)

acc = accuracy_score(Y_test, pred_test)

print("Test accuracy score : %s " % acc)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_1 (Bidirection (None, 187, 256)          133120    
_________________________________________________________________
bidirectional_2 (Bidirection (None, 187, 128)          164352    
_________________________________________________________________
bidirectional_3 (Bidirection (None, 64)                41216     
_________________________________________________________________
dense_1 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_2 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_3 (Dense)              (None, 5)                 165       
Total params: 345,093
Trainable params: 345,093
Non-trainable params: 0
_________________________________________________________________
Trai

KeyboardInterrupt: 